In [1]:
%load_ext autoreload
%autoreload 2

import torch
from torchmetrics import MeanSquaredError
import numpy as np
import pandas as pd
from dynaconf import Dynaconf
import lightning as L

from src.models.mf_with_bias import MatrixFactorizationWithBias
from src.lit_models.base import LightningModel

from src.datasets.movielens import MovielensDataModule

## Load config with dynaconf

In [3]:
cfg = Dynaconf(root_path="configs", settings_files=["config_mf.yaml"])

Load model inputs from the config file

In [4]:
n_users = cfg.model.pytorch_model.init_args.n_users
n_items = cfg.model.pytorch_model.init_args.n_items
n_factors = cfg.model.pytorch_model.init_args.n_factors
print(f"{n_users=}, {n_items=}, {n_factors=}")

n_users=943, n_items=1625, n_factors=128


## Load model

In [5]:
pytorch_model = MatrixFactorizationWithBias(n_users, n_items, n_factors)
pytorch_model

MatrixFactorizationWithBias(
  (user_emb): Embedding(943, 128)
  (user_bias): Embedding(943, 1)
  (item_emb): Embedding(1625, 128)
  (item_bias): Embedding(1625, 1)
)

In [63]:
checkpoint_file = "lightning_logs/embedding_dim/version_1/checkpoints/epoch=17-step=342.ckpt"

# load weights
model = LightningModel.load_from_checkpoint(
    checkpoint_path=checkpoint_file, pytorch_model=pytorch_model
)

In [8]:
device = model.device
print(device)

cpu


In [9]:
dm = MovielensDataModule(dataset="ml-100k", target="rating", batch_size=32)
dm.setup(stage="test")

## Predict on new data

Here we are going to use test set as a new data but, of course, we could use any new dataset.

In [10]:
test_dataloader = dm.test_dataloader()

In [54]:
# Get first batch of data
for batch_data in test_dataloader:
    users = batch_data["user"].to(device)
    items = batch_data["item"].to(device)
    ratings = batch_data["rating"].to(device)
    break

### Pytorch

We can make our predictions with plain Pytorch our using the Lighning Trainer.

Docs: https://lightning.ai/docs/pytorch/stable/deploy/production_intermediate.html

In [71]:
with torch.inference_mode():
    y_hat = model(users, items)  #* (5.5 - 1) + 1

y_hat

tensor([0.7605, 3.5300, 3.1920, 2.3851, 3.3966, 3.8786, 4.0043, 3.4172, 3.5833,
        1.6338, 3.8016, 2.5370, 2.3879, 3.8643, 3.3164, 3.4744, 3.5799, 2.3226,
        4.4144, 4.5998, 2.6984, 3.0930, 1.8979, 3.6119, 4.4471, 2.4540, 1.4081,
        3.8106, 2.0992, 2.9655, 2.4065, 3.1433])

Which is equivalent to call the `forward` of the pytorch model directly 

In [72]:
with torch.inference_mode():
    print(model.pytorch_model(users, items))

tensor([0.7605, 3.5300, 3.1920, 2.3851, 3.3966, 3.8786, 4.0043, 3.4172, 3.5833,
        1.6338, 3.8016, 2.5370, 2.3879, 3.8643, 3.3164, 3.4744, 3.5799, 2.3226,
        4.4144, 4.5998, 2.6984, 3.0930, 1.8979, 3.6119, 4.4471, 2.4540, 1.4081,
        3.8106, 2.0992, 2.9655, 2.4065, 3.1433])


In [73]:
# mse = MeanSquaredError().to(device)
# pred_list = []
# for batch_data in test_dataloader:
#     users = batch_data["user"].to(device)
#     items = batch_data["item"].to(device)
#     ratings = batch_data["rating"].to(device)
#     with torch.inference_mode():
#         y_hat = model.predict_step(batch_data) * (5.5 - 1) + 1
#         pred_list.append(y_hat.cpu().detach().numpy().squeeze())
    
#     mse(y_hat, ratings)

### Using Lightning Trainer

In [74]:
trainer = L.Trainer(enable_checkpointing=False)
batched_predictions = trainer.predict(model, dataloaders=[test_dataloader])

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Predicting DataLoader 0: 100%|██████████| 780/780 [00:01<00:00, 400.53it/s]


For comparison, show first batch fo predictions. Why are they different from the pure pytorch predictions? That's because in the LightningModule predictions are transformed using `Sigmoid` to normalize the oputpus.

In [76]:
batched_predictions[0]

tensor([0.6815, 0.9715, 0.9605, 0.9157, 0.9676, 0.9797, 0.9821, 0.9682, 0.9730,
        0.8367, 0.9782, 0.9267, 0.9159, 0.9795, 0.9650, 0.9700, 0.9729, 0.9107,
        0.9880, 0.9900, 0.9369, 0.9566, 0.8697, 0.9737, 0.9884, 0.9209, 0.8035,
        0.9783, 0.8908, 0.9510, 0.9173, 0.9586])

Concatenate all predictions in a single vector

In [77]:
predictions = torch.cat(batched_predictions)
predictions

tensor([0.6815, 0.9715, 0.9605,  ..., 0.9372, 0.9797, 0.9481])

Get true ratings

In [78]:
ratings = torch.cat([batch_data["rating"] for batch_data in test_dataloader], dim=0)
ratings

tensor([1., 2., 4.,  ..., 4., 5., 3.])

In [81]:
def scale_predictions(x, range=(1, 5.5)):
    min_y, max_y = range
    return x * (max_y - min_y) + min_y

In [82]:
mse = MeanSquaredError()
rmse = MeanSquaredError(squared=False)

test_mse = mse(scale_predictions(predictions), ratings)
test_rmse = rmse(scale_predictions(predictions), ratings)

print(f"Test MSE: {test_mse:.4f}")
print(f"Test RMSE: {torch.sqrt(test_rmse):.3f}")

Test MSE: 4.0108
Test RMSE: 1.415


In [84]:
test_mae = mse.compute()
print(f"Test MSE: {test_mse:.4f}")
print(f"Test RMSE: {torch.sqrt(test_mse):.3f}")

Test MSE: 4.0108
Test RMSE: 2.003
